In [1]:
import json
import requests
import asyncio
import websockets
import datetime
import requests


In [2]:
def get_all_events(closed="false", tag_id = ''):
    params = {
        "closed": closed,
        "limit" : 500,
        "offset" : 0,
        # 'tag_id': ''
    }
    if tag_id:
        params['tag_id'] = tag_id

    events = []
    r = requests.get(url="https://gamma-api.polymarket.com/events", params=params)
    response = r.json()
    while response:
        events += response
        params['offset'] += 500
        r = requests.get(url="https://gamma-api.polymarket.com/events", params=params)
        response = r.json()
    
    return events

def filter_crypto_events(events):
    crypto_events = []

    for event in events:
        if 'tags' not in event:
            print("No tags in", event['ticker'])
            continue
        tags = event['tags']
        for tag in tags:
            if tag['slug'] in ['crypto', 'crypto-prices', 'bitcoin', 'ethereum', 'stablecoins', 'solana', 'xrp']:
                crypto_events.append(event)
                break
    
    return crypto_events

def get_short_term_events(events):
    short_events = []
    for event in events:
        if 'tags' not in event:
            print("No tags in", event['ticker'])
            continue
        tags = event['tags']
        for tag in tags:
            if tag['slug'] in ['15M', '1H', '4h']:
                short_events.append(event)
                break
    return short_events

crypto_tag_id = '21'
events = get_all_events(tag_id=crypto_tag_id)
short_events = get_short_term_events(events)
# crypto_events = filter_crypto_events(events)

In [3]:
test = get_all_events(tag_id='102467')

In [29]:
test[3]

{'id': '67162',
 'ticker': 'eth-updown-15m-1761679800',
 'slug': 'eth-updown-15m-1761679800',
 'title': 'Ethereum Up or Down - October 28, 3:30PM-3:45PM ET',
 'description': 'This market will resolve to "Up" if the Ethereum price at the end of the time range specified in the title is greater than or equal to the price at the beginning of that range. Otherwise, it will resolve to "Down".\nThe resolution source for this market is information from Chainlink, specifically the ETH/USD data stream available at https://data.chain.link/streams/eth-usd.\nPlease note that this market is about the price according to Chainlink data stream ETH/USD, not according to other sources or spot markets.',
 'resolutionSource': 'https://data.chain.link/streams/eth-usd',
 'startDate': '2025-10-28T16:36:17.599359Z',
 'creationDate': '2025-10-28T16:36:17.599355Z',
 'endDate': '2025-10-28T19:45:00Z',
 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ETH+fullsize.jpg',
 'icon': 'https://polymarket-u

In [13]:
json.loads(test[0]['markets'][0]['clobTokenIds'])

['52214197939136034040737139916268185792854919672730843763012664188947714707029',
 '84456795910913157704978738144709424267516455881074352529687725153607043121125']

In [25]:
asset1, asset2 = json.loads(test[16]['markets'][0]['clobTokenIds'])
asset3, asset4 = json.loads(test[17]['markets'][0]['clobTokenIds'])

In [8]:
typetest

[{'id': '67105',
  'ticker': 'eth-updown-15m-1761677100',
  'slug': 'eth-updown-15m-1761677100',
  'title': 'Ethereum Up or Down - October 28, 2:45PM-3:00PM ET',
  'description': 'This market will resolve to "Up" if the Ethereum price at the end of the time range specified in the title is greater than or equal to the price at the beginning of that range. Otherwise, it will resolve to "Down".\nThe resolution source for this market is information from Chainlink, specifically the ETH/USD data stream available at https://data.chain.link/streams/eth-usd.\nPlease note that this market is about the price according to Chainlink data stream ETH/USD, not according to other sources or spot markets.',
  'resolutionSource': 'https://data.chain.link/streams/eth-usd',
  'startDate': '2025-10-28T15:52:55.833272Z',
  'creationDate': '2025-10-28T15:52:55.833269Z',
  'endDate': '2025-10-28T19:00:00Z',
  'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ETH+fullsize.jpg',
  'icon': 'https://p

In [4]:
for event in test:
    print(event['title'])
    print(event['markets'])

Ethereum Up or Down - October 28, 2:45PM-3:00PM ET
[{'id': '653861', 'question': 'Ethereum Up or Down - October 28, 2:45PM-3:00PM ET', 'conditionId': '0x92f3a42b1ee86e4ee2e81877078a7e12ae5f64da0f732e57896865810e828b59', 'slug': 'eth-updown-15m-1761677100', 'resolutionSource': 'https://data.chain.link/streams/eth-usd', 'endDate': '2025-10-28T19:00:00Z', 'liquidity': '8174.2119', 'startDate': '2025-10-28T15:47:15.754481Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ETH+fullsize.jpg', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/ETH+fullsize.jpg', 'description': 'This market will resolve to "Up" if the Ethereum price at the end of the time range specified in the title is greater than or equal to the price at the beginning of that range. Otherwise, it will resolve to "Down".\nThe resolution source for this market is information from Chainlink, specifically the ETH/USD data stream available at https://data.chain.link/streams/eth-usd.\nPlease note that th

In [5]:
condition_id = "0x56f8de8878e825b73937be862fdcd7af2497c5b73df1dac0fd9113bf11846176"
def get_market_trades(condition_id = condition_id):
    url = "https://data-api.polymarket.com/trades"
    params = {
        "limit": 500,
        "offset": 0,
        "market": [condition_id],
        # "takerOnly": "true",
    }
    
    total_trades = []
    response = requests.get(url, params=params)
    cur_trades = response.json()

    while cur_trades:
        total_trades += cur_trades
        params['offset'] += 500
        response = requests.get(url, params=params)
        cur_trades = response.json()

    return total_trades
    

In [6]:
market_trades = get_market_trades(condition_id)

In [17]:
market_trades

[{'proxyWallet': '0x47a35c8fdcdce37d9e19241d34e295fcb4911bce',
  'side': 'SELL',
  'asset': '101160920341113437254721927007904783071619510617577160798815803852851616980409',
  'conditionId': '0x56f8de8878e825b73937be862fdcd7af2497c5b73df1dac0fd9113bf11846176',
  'size': 1.06,
  'price': 0.99,
  'timestamp': 1761653725,
  'title': 'Bitcoin Up or Down - October 28, 8:00AM-8:15AM ET',
  'slug': 'btc-updown-15m-1761652800',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/BTC+fullsize.png',
  'eventSlug': 'btc-updown-15m-1761652800',
  'outcome': 'Down',
  'outcomeIndex': 1,
  'name': 'kkk829',
  'pseudonym': 'Excited-Choosing',
  'bio': '',
  'profileImage': '',
  'profileImageOptimized': '',
  'transactionHash': '0xb7b6e6d54711e0e0ec88596ac3398f496a19e7118a597fe9b85d95140b9e5906'},
 {'proxyWallet': '0xd13321a9af3ecb5fa33f9a36fc368c783c81d55f',
  'side': 'SELL',
  'asset': '101160920341113437254721927007904783071619510617577160798815803852851616980409',
  'conditionId': '0x

In [7]:
# ATTEMPTING TO SUBSCRIBE TO CHAINLINK DATA
async def stream_chainlink_data():
  url = "wss://ws-live-data.polymarket.com"
  last_time_pong = datetime.datetime.now()
  subscribe_message = {
      "action": "subscribe",
      "subscriptions": [
        {
          "topic": "crypto_prices_chainlink",
          "type": "*",
          "filters": "{\"symbol\":\"btc/usd\"}"
        }
      ]
    }
  crypto_msgs = []
  try:
    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps(subscribe_message))

        while True:
            m = await websocket.recv()
            if m != "PONG":
                last_time_pong = datetime.datetime.now()
            try:
                d = json.loads(m)
            except json.JSONDecodeError:
                print("Received non-JSON message:", m)
                continue
            print(d)
            if last_time_pong + datetime.timedelta(seconds=5) < datetime.datetime.now():
                await websocket.send("PING")
            else:
                crypto_msgs.append(d)
  finally:
    await websocket.close

In [9]:
task.cancel()

True

In [1]:
task = asyncio.create_task(stream_chainlink_data())

NameError: name 'asyncio' is not defined

In [ ]:
start_time = market['eventStartTime']
end_time = market['endDate']
market_start_utc = dt.datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%S%z')
market_end_utc = dt.datetime.strptime(end_time, '%Y-%m-%dT%H:%M:%S%z')
market_start_utc.timestamp(), market_end_utc.timestamp()

In [ ]:
import json

url = 'wss://ws-subscriptions-clob.polymarket.com/ws/market'
last_time_pong = datetime.datetime.now()
msgs = []

# asset_ids = [yes_token, no_token]
async def stream_market_data(url, asset_ids: list, ):

    msgs = []
    last_time_pong = datetime.datetime.now()

    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps({"assets_ids":asset_ids,"type":"market"}))

        while True:
            m = await websocket.recv()
            if m != "PONG":
                last_time_pong = datetime.datetime.now()
            try:
                d = json.loads(m)
            except json.JSONDecodeError:
                print("Received non-JSON message:", m)
                continue
            print(d)
            if last_time_pong + datetime.timedelta(seconds=10) < datetime.datetime.now():
                await websocket.send("PING")
            else:
                msgs.append(d)
        
    

In [28]:
asset_ids = [asset1, asset2, asset3, asset4]
url = 'wss://ws-subscriptions-clob.polymarket.com/ws/market'
msgs = []
async with websockets.connect(url) as websocket:
    await websocket.send(json.dumps({"assets_ids":asset_ids[:1],"type":"market"}))
    await asyncio.sleep(2)
    await websocket.send(json.dumps({"assets_ids":asset_ids[1:2],"type":"market"}))

    while len(msgs) < 100:
        m = await websocket.recv()
        if m != "PONG":
            last_time_pong = datetime.datetime.now()
        try:
            d = json.loads(m)
            msgs.append(d)
            print(d)
        except json.JSONDecodeError:
            print("Received non-JSON message:", m)
            continue
        
        if last_time_pong + datetime.timedelta(seconds=10) < datetime.datetime.now():
            await websocket.send("PING")
        # else:
        #     pass
        if len(msgs) > 50:
            print("UNSUBSCRIBING FROM 1")
            await websocket.send(json.dumps({"assets_ids":asset_ids[1:2],"type":"market"}))
            # msgs.append(d)
    
    
        


[{'market': '0x34ba0a0848ce5f77566aa170921a587f1ab2efb634d4eaa2408d554ced0cd3d1', 'asset_id': '34203708354223716468993269493197618933777771310905853151982894863671565995929', 'timestamp': '1761685171017', 'hash': 'ea13bd0beba16d17bed099ff00edcadd170ad8eb', 'bids': [{'price': '0.01', 'size': '24251'}, {'price': '0.02', 'size': '300'}, {'price': '0.03', 'size': '10'}, {'price': '0.04', 'size': '500'}, {'price': '0.05', 'size': '1000'}, {'price': '0.06', 'size': '4000'}, {'price': '0.07', 'size': '500'}, {'price': '0.08', 'size': '300'}, {'price': '0.1', 'size': '3000'}], 'asks': [{'price': '0.99', 'size': '24251'}, {'price': '0.98', 'size': '300'}, {'price': '0.97', 'size': '10'}, {'price': '0.96', 'size': '500'}, {'price': '0.95', 'size': '1000'}, {'price': '0.94', 'size': '4000'}, {'price': '0.93', 'size': '500'}, {'price': '0.92', 'size': '300'}, {'price': '0.9', 'size': '3000'}], 'event_type': 'book', 'last_trade_price': ''}]
Received non-JSON message: PONG
{'market': '0x34ba0a0848ce

CancelledError: 

In [ ]:
class PolyMarketStream:
    def __init__(self, url='wss://ws-subscriptions-clob.polymarket.com/ws/market'):
        self.url = url
        self.asset_ids = set()
        self.last_pong = datetime.datetime.now()
        self.websocket = None
        self.keep_running = True

        # Queues for external communication
        self.command_queue = asyncio.Queue()  # for subscribe/unsubscribe
        self.data_queue = asyncio.Queue()     # for received market messages

    async def connect(self):
        async with websockets.connect(self.url) as ws:
            self.websocket = ws
            print("✅ Connected to Polymarket WebSocket")

            # send initial subscription (if any)
            if self.asset_ids:
                await self._send_subscribe()

            # run tasks concurrently
            await asyncio.gather(
                self._receiver(),
                self._heartbeat(),
                self._command_listener()
            )

    async def _receiver(self):
        """Receives market data and PONGs."""
        try:
            async for msg in self.websocket:
                if msg == "PONG":
                    self.last_pong = datetime.datetime.now()
                    continue

                data = json.loads(msg)
                await self.data_queue.put(data)
        except Exception as e:
            print(f"❌ Receiver error: {e}")

    async def _heartbeat(self):
        """Sends PING every 10 seconds to keep the connection alive."""
        while self.keep_running:
            if (datetime.datetime.now() - self.last_pong).seconds > 5:
                await self.websocket.send("PING")
            await asyncio.sleep(5)

    async def _command_listener(self):
        """Listens for subscribe/unsubscribe commands from the main loop."""
        while self.keep_running:
            cmd, asset_id = await self.command_queue.get()

            if cmd == "subscribe":
                self.asset_ids.add(asset_id)
                await self._send_subscribe()

            elif cmd == "unsubscribe":
                self.asset_ids.discard(asset_id)
                await self._send_unsubscribe()

            elif cmd == "close":
                self.keep_running = False
                await self.websocket.close()
                break

    async def _send_subscribe(self):
        msg = {"type": "market", "assets_ids": list(self.asset_ids)}
        await self.websocket.send(json.dumps(msg))
        print(f"📡 Subscribed to {self.asset_ids}")

    async def _send_unsubscribe(self):
        msg = {"type": "unsubscribe", "assets_ids": list(self.asset_ids)}
        await self.websocket.send(json.dumps(msg))
        print(f"❎ Unsubscribed from {self.asset_ids}")

    async def subscribe(self, asset_id):
        await self.command_queue.put(("subscribe", asset_id))

    async def unsubscribe(self, asset_id):
        await self.command_queue.put(("unsubscribe", asset_id))

    async def next_message(self):
        """Allows external code to get the next message."""
        return await self.data_queue.get()

    async def close(self):
        await self.command_queue.put(("close", None))


async def main():
    stream = PolyMarketStream()

    # start the connection
    asyncio.create_task(stream.connect())

    # subscribe dynamically
    await asyncio.sleep(1)
    await stream.subscribe("1234")
    await stream.subscribe("5678")

    # process incoming data
    while True:
        msg = await stream.next_message()
        print("📈 Market update:", msg)

        # you can decide to unsubscribe dynamically
        # e.g. after a certain condition
        # await stream.unsubscribe("1234")

# Run it
asyncio.run(main())


In [ ]:
filename = "btc_8_815_updown.json"
with open(filename, "w") as f:
    json.dump(msgs, f, indent=4)